In [1]:
from sequential_tasks import TemporalOrderExp6aSequence

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Specify experiment settings ands prepare the data

In [2]:
# experiments settings
settings = {
    "difficulty": TemporalOrderExp6aSequence.DifficultyLevel.EASY,
    "batch_size": 32,
    "h_units": 4,
    "max_epochs": 30
}

In [3]:
#training data
train_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
    settings['difficulty'],
    settings['batch_size'])
train_size = len(train_data_gen)

# testing data
test_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
    settings['difficulty'],
    settings['batch_size'])
test_size = len(test_data_gen)   

# Define neural network

In [4]:
class SimpleRNN(nn.Module):

    def __init__(self, input_size, rnn_hidden_size, output_size):

        super(SimpleRNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size, rnn_hidden_size, num_layers=1, nonlinearity='relu', batch_first=True)
        self.linear = torch.nn.Linear(rnn_hidden_size, output_size) 

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.linear(x)
        return F.log_softmax(x, dim=1)

# Define training loop

In [5]:
def train():
    model.train()
    correct = 0
    for batch_idx in range(train_size):
        data, target = train_data_gen[batch_idx]
        data, target = torch.from_numpy(data).float(), torch.from_numpy(target).long()
        optimizer.zero_grad()
        y_pred = model(data)
        loss = criterion(y_pred, target)
        loss.backward()
        optimizer.step()
        
        pred = y_pred.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
    return correct, loss    

In [6]:
def test():
    model.eval()   
    correct = 0
    with torch.no_grad():
        for batch_idx in range(test_size):
            data, target = test_data_gen[batch_idx]
            data, target = torch.from_numpy(data).float(), torch.from_numpy(target).long()
            y_pred = model(data)
            pred = y_pred.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct

# Initialize the Model and Optimizer

In [7]:
model = SimpleRNN(train_data_gen.n_symbols, settings['h_units'], train_data_gen.n_classes)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

# Train the model

In [8]:
#train for max_epochs epochs
epochs = settings['max_epochs']
epoch = 0
while epoch < epochs:
    correct, loss = train()

    epoch += 1
    train_accuracy = float(correct) / train_size
    print('Train Epoch: {}/{}, loss: {:.4f}, accuracy {:2.2f}'.format(epoch, epochs, loss.item(), train_accuracy))

#test    
correct = test()
test_accuracy = float(correct) / test_size
print('\nTest accuracy: {}'.format(test_accuracy))

Train Epoch: 1/30, loss: 2.1147, accuracy 27.90
Train Epoch: 2/30, loss: 1.9919, accuracy 53.71
Train Epoch: 3/30, loss: 1.7164, accuracy 72.97
Train Epoch: 4/30, loss: 1.2584, accuracy 94.13
Train Epoch: 5/30, loss: 0.9046, accuracy 96.00
Train Epoch: 6/30, loss: 0.7676, accuracy 96.00
Train Epoch: 7/30, loss: 0.6871, accuracy 96.00
Train Epoch: 8/30, loss: 0.6495, accuracy 96.00
Train Epoch: 9/30, loss: 0.6394, accuracy 96.00
Train Epoch: 10/30, loss: 0.6372, accuracy 96.00
Train Epoch: 11/30, loss: 0.6069, accuracy 96.00
Train Epoch: 12/30, loss: 0.6126, accuracy 96.00
Train Epoch: 13/30, loss: 0.5870, accuracy 96.00
Train Epoch: 14/30, loss: 0.5999, accuracy 96.00
Train Epoch: 15/30, loss: 0.5698, accuracy 96.00
Train Epoch: 16/30, loss: 0.5882, accuracy 96.00
Train Epoch: 17/30, loss: 0.5850, accuracy 96.00
Train Epoch: 18/30, loss: 0.5813, accuracy 96.00
Train Epoch: 19/30, loss: 0.5703, accuracy 96.00
Train Epoch: 20/30, loss: 0.5754, accuracy 96.00
Train Epoch: 21/30, loss: 0.5

In [25]:
print('acc = {:.2f}%.'.format(test_accuracy))

acc = 96.00%.
